In [2]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel


In [6]:
from transformers import RobertaModel, RobertaTokenizer

# Load the model
model_name = "roberta-large"
model = RobertaModel.from_pretrained(model_name)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
df = pd.read_csv('../Data/wiki-labeled.csv')


In [9]:
gpt = df[df['label'] == 0]
human = df[df['label'] == 1]

In [10]:
gpt = gpt.sample(n=150)
human = human.sample(n=150)

In [11]:
def generate_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        #embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        last_hidden_states = outputs.last_hidden_state
        embeddings = last_hidden_states[:,0,:]
    return embeddings.numpy()


In [12]:
import numpy as np
def remove_outliers(embeddings):
        # Calculate the mode of the embeddings
        mode_val = np.mean(embeddings[0], axis=None)
        
        # Replace outliers with the mode value
        embeddings[np.abs(embeddings - mode_val) > 2 * np.std(embeddings)] = mode_val
        
        return embeddings
def min_max_normalize(embeddings):
    # Find the minimum and maximum values in the embeddings
    min_val = np.min(embeddings)
    max_val = np.max(embeddings)
    
    # Normalize the embeddings to range [0, 255]
    normalized_embeddings = 255 * (embeddings - min_val) / (max_val - min_val)
    
    return normalized_embeddings.astype(np.uint8)  # Convert to uint8 for integer values between 0 and 255

def reshape_embeddings(embeddings):
    # Reshape the embeddings to 3D array
    return embeddings.reshape(32,32)

In [13]:
gpt['embeddings'] = gpt['text'].apply(generate_bert_embeddings)
gpt['normalized_embeddings'] = gpt['embeddings'].apply(remove_outliers)
gpt['normalized_embeddings1'] = gpt['normalized_embeddings'].apply(min_max_normalize)
gpt['reshaped_embeddings'] = gpt['normalized_embeddings1'].apply(reshape_embeddings)

In [14]:
human['embeddings'] = human['text'].apply(generate_bert_embeddings)
human['normalized_embeddings'] = human['embeddings'].apply(remove_outliers)
human['normalized_embeddings1'] = human['normalized_embeddings'].apply(min_max_normalize)
human['reshaped_embeddings'] = human['normalized_embeddings1'].apply(reshape_embeddings)

In [3]:
gpt_embed = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/gpt_embeddings_wiki.npy', allow_pickle=True)

In [4]:
human_embed = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/human_embeddings_wiki.npy', allow_pickle=True)

In [5]:
gpt_embed1 = list(gpt_embed)
human_embed1 = list(human_embed)

In [28]:
gpt_embed = torch.tensor(gpt['normalized_embeddings'].tolist())

/tmp/ipykernel_596170/3765168792.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/torch/csrc/utils/tensor_new.cpp:245.)
  gpt_embed = torch.tensor(gpt['normalized_embeddings'].tolist())


In [31]:
human_embed = torch.tensor(human['normalized_embeddings'].tolist())

In [6]:
gpt_embed_tensor = torch.tensor(gpt_embed1)
human_embed_tensor = torch.tensor(human_embed1)

/tmp/ipykernel_605992/1006583565.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/torch/csrc/utils/tensor_new.cpp:245.)
  gpt_embed_tensor = torch.tensor(gpt_embed1)


In [29]:
print((gpt_embed.dtype))

torch.float32


In [30]:
print(gpt_embed[0].shape)

torch.Size([1, 1024])


In [7]:
import torch

# Create tensors with zeros and ones
zeros_tensor = torch.zeros(15000)
ones_tensor = torch.ones(15000)

# Concatenate the tensors along the first dimension
labels = torch.cat((zeros_tensor, ones_tensor), dim=0)

# Display the result tensor
print(labels)


tensor([0., 0., 0.,  ..., 1., 1., 1.])


In [8]:
embeds = torch.cat((gpt_embed_tensor, human_embed_tensor), dim=0)

In [9]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

test_size = 0.2
dataset = TensorDataset(embeds, labels)

# Split dataset into train and test sets
train_dataset, test_dataset = train_test_split(dataset, test_size=test_size, random_state=42)

# Define batch size
batch_size = 32

# Create DataLoader for training set
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv1d(in_channels, 8, kernel_size=16, stride=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=8, stride=2)

        self.conv2 = nn.Conv1d(8, 16, kernel_size=8, stride=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=4, stride=2)

        # self.conv3 = nn.Conv2d(4, 128, kernel_size=3, stride=1, padding=1)
        # self.relu3 = nn.ReLU()
        # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(1920, 512)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(512, 128)
        self.relu5 = nn.ReLU()

        self.fc3 = nn.Linear(128, 2)
        

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        # x = self.pool3(self.relu3(self.conv3(x)))
        # print(x.shape)
        x = self.flatten(x)
        # print(x.shape)
        x = self.relu4(self.fc1(x))
        x = self.relu5(self.fc2(x))
        x = self.fc3(x) 

        return x


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(in_channels=1, num_classes=2).to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-2)

# Step 5: Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        labels = labels.long()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_dataloader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, 1)
            total += labels.size(0)
            #print(labels, predicted)
            correct += (predicted == labels).sum().item()
    
    val_accuracy = correct / total
    print(f'Validation Accuracy: {val_accuracy:.4f}')

Epoch [1/100], Loss: 0.1498
Validation Accuracy: 0.9838
Epoch [2/100], Loss: 0.0630
Validation Accuracy: 0.9883
Epoch [3/100], Loss: 0.0442
Validation Accuracy: 0.9863
Epoch [4/100], Loss: 0.0395
Validation Accuracy: 0.9792
Epoch [5/100], Loss: 0.0396
Validation Accuracy: 0.9907
Epoch [6/100], Loss: 0.0350
Validation Accuracy: 0.9898
Epoch [7/100], Loss: 0.0319
Validation Accuracy: 0.9827
Epoch [8/100], Loss: 0.0294
Validation Accuracy: 0.9882
Epoch [9/100], Loss: 0.0333
Validation Accuracy: 0.9903
Epoch [10/100], Loss: 0.0287
Validation Accuracy: 0.9935
Epoch [11/100], Loss: 0.0253
Validation Accuracy: 0.9922
Epoch [12/100], Loss: 0.0277
Validation Accuracy: 0.9923
Epoch [13/100], Loss: 0.0243
Validation Accuracy: 0.9923
Epoch [14/100], Loss: 0.0233
Validation Accuracy: 0.9820
Epoch [15/100], Loss: 0.0263
Validation Accuracy: 0.9908
Epoch [16/100], Loss: 0.0236
Validation Accuracy: 0.9945
Epoch [17/100], Loss: 0.0199
Validation Accuracy: 0.9900
Epoch [18/100], Loss: 0.0203
Validation 

In [13]:
import torch
import torchvision.models as models

model_path = "/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/model/wiki_model_1D.pth"

torch.save(model.state_dict(), model_path)
